# Evaluate LLM results

Install Dependencies

In [1]:
# %%capture
# !pip install datasets==2.20.0
# !pip install -U langsmith==0.1.99
# !pip install langchain_openai==0.2.0 #0.1.22
# !pip install langchain==0.3.0 #0.2.13
# !pip install langchain_community==0.3.0 #0.2.12  
# !pip install langchain-huggingface==0.1.0                      
# !pip install transformers==4.44.0
# !pip install torch==2.1.0
# !pip install termcolor==2.4.0
# !pip install accelerate==0.33.0
# !pip install pandas==2.2.2
# !pip install openpyxl==3.1.5
# !pip install python-dotenv==1.0.1
# !pip install einops==0.8.0
# !pip install wheel==0.44.0
# !pip install sentencepiece==0.2.0
# !pip install protobuf==5.27.3 #Mistral models needs this
# !pip install groq==0.10.0 #Groq models needs this
# !pip install matplotlib==3.9.2
# !pip install seaborn==0.13.2
# !pip install scipy==1.14.1
# !pip install statsmodels==0.14.4
# !pip install anthropic==0.40.0 #Anthropic models needs this
# !pip install together==1.3.14 #Together models needs this
# !pip install google-generativeai==0.8.4
# !pip install google-genai==0.8.0
# !pip install sentence-transformers==3.3.1

# !pip install flash-attn==2.6.3 #Install it at the end after wheel has been installed

# #Only if CPU is used
# !pip install torch==2.0.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
# !jupyter lab --ServerApp.iopub_data_rate_limit=1e10

In [3]:
import warnings
warnings.filterwarnings('ignore')

RunPod specific parameters

In [4]:
#For RunPod change to persistent storage directory
import os
os.chdir('/workspace')

Specify Path and Load API Keys

In [5]:
file_path ='/workspace/DRACO_draft.xlsx' #Dataset generated with the help of GPT-4o - Has to be an excel file with 'input' and 'output' columns
#'/Users/nikolaossourlo/Desktop/Example_QA_data_raw.xlsx' #For MacOS
#'C:/Users/soyrl/Desktop/Example_QA_data_raw.xlsx' #For Windows
#'/content/drive/My Drive/Example_QA_data_raw.xlsx' #For Google Colab
#'/home/nikolaossourlo/Example_QA_data_raw.xlsx' #For Delft Blue
#'/workspace/Example_QA_data_raw.xlsx' #For RunPod

custom_cache_dir="/workspace/cache/huggingface" #Save models here so that we don't have to download them again
#"/scratch/nikolaossourlo/cache" in Delft Blue

# Check if custom_cache_dir is defined, otherwise use default behavior
try:
    cache_dir=custom_cache_dir
except:
    cache_dir=None

from dotenv import load_dotenv
import os
import json
import numpy as np
import traceback

# Load environment variables from .env file
load_dotenv(dotenv_path=os.getcwd()+"/env")

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY_DRACO')
langsmith_api_key = os.getenv('LANGSMITH_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY_DRACO')
together_api_key = os.getenv('TOGETHER_API_KEY_DRACO')
open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')

#Login to Hugging Face
from huggingface_hub import login
# Log in with your Hugging Face token
login(token=os.getenv('HF_TOKEN'))

# print(openai_api_key)
# print(langsmith_api_key)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Select model and name for the experiment

In [6]:
#Model to generate responses to questions - Sometimes we might have to restart session and comment out the models that have already been run
models=[ #Reasoning models (deepseek only for now) should have longer max_tokens to include the reasoning steps
    # "together/Qwen/QwQ-32B",     # "Qwen/QwQ-32B-AWQ",
    # "together/deepseek-ai/DeepSeek-R1",
    # "openai/o1", #o1-2024-12-17
    # "openai/o1-mini", #o1-mini-2024-09-12
    # "openai/o3-mini", #o3-mini-2025-01-31
    # "openai/gpt-4o-2024-08-06",
    # "gemini/gemini-2.0-flash-exp",
    # "together/deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
    # "together/meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    # "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    # 'microsoft/phi-4', #14B parameters

    # "Qwen/Qwen2.5-7B-Instruct",
    # "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Meta-Llama-3.1-8B-Instruct", #A4500 (20GB VRAM) and in Delft Blue (V100 32GB)
    # "microsoft/Phi-3.5-mini-instruct", #A40 with 48GB VRAM, A4500 with 20GB VRAM, Delft Blue 
    # "mistralai/Mistral-7B-Instruct-v0.3", #A40 with 48GB VRAM, A4500 with 20GB VRAM and in Delft Blue
    # "Qwen/Qwen2-7B-Instruct", #A40 with 48GB VRAM, A4500 with 20GB VRAM, Delft Blue
    # 'AI-MO/NuminaMath-7B-TIR', #A4500 with 20GB VRAM and in Delft Blue - We can also try 01-ai/Yi-Coder-9B-Chat
    # 'microsoft/Phi-3-mini-4k-instruct', #RTX3090
    # "google/gemma-2-9b-it", #More than 20GB of GPU memory needed - Works with A40 with 48GB VRAM, but not with A4500 - 20GB, and V100 - 32GB, Delft Blue
    # 'mistralai/Mistral-Nemo-Instruct-2407', #12B parameters, 2 RTX3090, V100 with 32GB VRAM
    # "anthropic/claude-3-5-sonnet-20241022",
    # 'openai/gpt-4o-mini' #Costs very low ~0.01$ for 9 Q&A pairs.
    # "gemini/gemini-2.0-flash-thinking-exp", #Thoughts only in Google studio, not in API - https://discuss.ai.google.dev/t/thoughts-are-missing-cot-not-included-anymore/63653/8
    ] #Takes 7+hours in A40 for the above 13 models with 7Q&A paris and 4 resamples. Cost ±3$ (±180GB)

# Groq models are defined as: groq_website/model_name e.g. 'groq_website/llama-3.1-70b-versatile'
# OpenAI models are defined as: 'openai/model_name', e.g. 'openai/gpt-4o-mini'
# Anthropic models are defined as 'anthropic/model_name', e.g. 'anthropic/claude-3-haiku-20240307' - Couldn't use due to billing issues
# Together models are defined as 'together/model_name', e.g. 'together/meta-llama/Llama-3.3-70B-Instruct-Turbo-Free'
# OpenRouter models are defined as 'openrouter/model_name', e.g. 'openrouter/deepseek/deepseek-r1:free' - Do not work due to extremely limited quota
# Gemini models are defined as 'gemini/model_name', e.g. 'gemini/gemini-2.0-flash-exp'

# I couldn't run 'nvidia/Mistral-NeMo-Minitron-8B-Base', "hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4" (Conflicting dependencies),
# 'google/recurrentgemma-9b-it' # RecurrentGemmaForCausalLM.forward() got an unexpected keyword argument 'position_ids'
#Large models take more time (2min/generation for Mistral 12B)

#Define model to act as a judge
judge_model='openai/gpt-4o-mini' #If used with Llama, only 0.01$ for 9 Q&A pairs for gpt-4o-mini, and 0.22$ for gpt-4o

#Used below to distinguish commercial and Hugging Face models
commercial_api_providers=['openai','groq_website','anthropic','together', 'openrouter', 'gemini']

#Define maximum number of tokes in the judge LLM output
max_output_tokens=500

#Limit of tokens in the generated response from LLM - For reasoning models we increase it to 4000 - had to define it below. 
generate_max_tokens=1000

#Domain - Chemical/Water Engineering or anything else
domain="Water"

#Inference on whole dataset?
inference_on_whole_dataset=True

#Number of times to resample the dataset
n_resamples=1 #4 reduces the variance to 50%

Define prompts for custom evaluation metrics

In [7]:
common_prompt=""" 
You are an autoregressive language model that acts as a judge in comparing a predicted vs an actual answer to a questions.
Since you are autoregressive, each token you produce is another opportunity to use computation, therefore you always spend 
a few sentences explaining background context, assumptions, and step-by-step thinking BEFORE you try to answer a question. 
Your users are experts in"""+ domain +"""engineering, so they already know you're a language model and your capabilities and limitations, so don't 
remind them of that. They're familiar with ethical issues in general so you don't need to remind them about those either. 
Don't be verbose in your answers, but do provide details and examples where it might help the explanation. 
""" #This is common for all prompts below - change with 'chemical engineering' for chemical engineer questions

In [10]:
completeness_descr = """
Your task is to evaluate responses predicted by an LLM with regards to completeness compared to the completeness of a given actual, golden standard answer. 
The completeness metric evaluates the extent to which the user's question is answered in full in the predicted response. 
You can assign a score from 1 to 5 to the predicted response with the following interpretations:
1: There is no response.
2: No parts of a suitable answer are present.
3: Few elements of a complete answer are present.
4: Most elements of a complete answer are present.
5: The response covers all elements of a complete answer.
IMPORTANT: End your responses with the sentence: "FINAL SCORE:" followed by whole numbers only (1, 2, 3, 4, or 5). Do not use decimal points. This may not be disregarded!
"""

relevance_descr = """
Your task is to evaluate responses predicted by an LLM with regards to relevance compared to the relevance of a given actual, golden standard answer. 
The relevance metric evaluates the amount of irrelevant information in the predicted response considering the user's original question. 
You can assign a score from 1 to 5 to the predicted response with the following interpretations:
1: The response answers something else, not the user's question.
2: The response answers the user's question but the information provided is mostly irrelevant.
3: The response answers the user's question but contains more irrelevant information than relevant information.
4: The response answers the user's question, and shares a bit of irrelevant information.
5: The response answers the user's question and contains no irrelevant information.
IMPORTANT: End your responses with the sentence: "FINAL SCORE:" followed by whole numbers only (1, 2, 3, 4, or 5). Do not use decimal points. This may not be disregarded!
"""

conciseness_descr = """
Your task is to evaluate responses predicted by an LLM with regards to conciseness compared to the conciseness of a given actual, golden standard answer. 
The conciseness metric evaluates the amount of unexpected extra information in the predicted response considering the user's original question. 
You can assign a score from 1 to 5 to the predicted response with the following interpretations:
1: The response is too long and stops before completion or enters an infinite loop.
2: The response includes a lot of extra information and uses flowery language.
3: The response includes a lot of extra information or uses flowery language.
4: The response is short and includes a small amount of extra information.
5: The response is as short as possible while still answering the prompt.
IMPORTANT: End your responses with the sentence: "FINAL SCORE:" followed by whole numbers only (1, 2, 3, 4, or 5). Do not use decimal points. This may not be disregarded!
"""

confidence_descr = """
Your task is to evaluate responses predicted by an LLM with regards to confidence compared to the confidence of a given actual, golden standard answer. 
The condifence metric evaluates the degree of assurance that is conveyed the response that the predicted answer is correct. 
You can assign a score from 1 to 5 to the predicted response with the following interpretations:
1: Complete Rejection. The response makes it clear that the given answer is incorrect or that no correct answer can be provided.
2: Doubt and Disagreement. The response suggests that the answer is likely incorrect or raises significant concerns.
3: Uncertainty. The response indicates that the answer could be correct, but there is significant doubt or insufficient evidence.
4: Moderate Agreement. The response leans towards the answer being correct but acknowledges some uncertainty.
5: Full Endorsement. The reponse confidentely asserts that the given answer is correct.
IMPORTANT: End your responses with the sentence: "FINAL SCORE:" followed by whole numbers only (1, 2, 3, 4, or 5). Do not use decimal points. This may not be disregarded!
"""

factuality_descr = """
Your task is to evaluate responses predicted by an LLM with regards to factuality compared to the factuality of a given actual, golden standard answer.
 The factuality metric evaluates the degree of hallucination contained in a response or, in other words, how accurate a given response is.
You can assign a score from 1 to 5, with the following interpretations:
1: The response is a complete hallucination
2: The response is mostly a hallucination but does not change key information from the prompt
3: The response contains large amounts of both hallucinations and factual information.
4: The response includes mostly factual information with slight hallucinations.
5: The response only includes factual information.
IMPORTANT: End your responses with the sentence: "FINAL SCORE:" followed by whole numbers only (1, 2, 3, 4, or 5). Do not use decimal points. This may not be disregarded!
""" #2: The response is mostly a hallucination but does not change key information from the prompt"""+ # (such as chemical identifiers).


judgement_descr = """
Your task is to evaluate responses predicted by an LLM with regards to judgement compared to the judgement of a given actual, golden standard answer.
The judgment metric assesses how strongly the response implies its correctness, taking into account the actual accuracy of the answer.
You can assign a score from 1 to 5 to the predicted response with the following interpretations:
1: The response confidently claims a hallucination as truth.
2: The response misinterprets information received in the prompt.
3: The response shows that the model is unsure about the answer or states that information is theoretical.
4: The response is wrong but it is made clear that the answer is wrong or that the model is unable to provide a correct answer.
5: The response is correct.
IMPORTANT: End your responses with the sentence: "FINAL SCORE:" followed by whole numbers only (1, 2, 3, 4, or 5). Do not use decimal points. This may not be disregarded!
"""

In [ ]:
# !pip install torch 
# !pip install mistral_inference

# !pip install mistral-common

In [11]:
from mistral_common.protocol.instruct.tool_calls import Function, Tool

# Define a new tool
tool_definitions = [
    # Tool(
    # function=Function(
    #     name="your_function_name",
    #     description="Describe what this function does",
    #     parameters={
    #         "type": "object",
    #         "properties": {
    #             "param1": {
    #                 "type": "string",
    #                 "description": "Description of param1",
    #             },
    #             "param2": {
    #                 "type": "number",
    #                 "description": "Description of param2",
    #             },
    #         },
    #         "required": ["param1", "param2"],
    #     },
    # )
    # ),
    Tool(
    function=Function(
        name="add",
        description="Adds two numbers together",
        parameters={
            "type": "object",
            "properties": {
                "number1": {
                    "type": "number",
                    "description": "The first number to add",
                },
                "number2": {
                    "type": "number",
                    "description": "The second number to add",
                },
            },
            "required": ["number1", "number2"],
        },
    )
    ),
    # Tool(
    # function=Function(
    #     name="subtract",
    #     description="Subtracts the second number from the first number",
    #     parameters={
    #         "type": "object",
    #         "properties": {
    #             "number1": {
    #                 "type": "number",
    #                 "description": "The number from which to subtract",
    #             },
    #             "number2": {
    #                 "type": "number",
    #                 "description": "The number to subtract",
    #             },
    #         },
    #         "required": ["number1", "number2"],
    #     },
    # )
    # ),
    # Tool(
    # function=Function(
    #     name="multiply",
    #     description="Multiplies two numbers together",
    #     parameters={
    #         "type": "object",
    #         "properties": {
    #             "number1": {
    #                 "type": "number",
    #                 "description": "The first number to multiply",
    #             },
    #             "number2": {
    #                 "type": "number",
    #                 "description": "The second number to multiply",
    #             },
    #         },
    #         "required": ["number1", "number2"],
    #     },
    # )
    # ),
    # Tool(
    # function=Function(
    #     name="exponential",
    #     description="Calculates the exponential of the first number raised to the power of the second number",
    #     parameters={
    #         "type": "object",
    #         "properties": {
    #             "base": {
    #                 "type": "number",
    #                 "description": "The base number",
    #             },
    #             "exponent": {
    #                 "type": "number",
    #                 "description": "The exponent to raise the base to",
    #             },
    #         },
    #         "required": ["base", "exponent"],
    #     },
    # )
    # ),
    Tool(
    function=Function(
        name="extract_code",
        description="Determines whether the provided text contains code. Use this tool whenever a user asks for a function, script, or programming-related response.",
        parameters={
            "type": "object",
            "properties": {
                "model_output": {
                    "type": "string",
                    "description": "The text to analyze for the presence of code.",
                },
            },
            "required": ["model_output"],
        },
    )
    ),

]

In [1]:
tool_definitions = [
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Adds two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "The first number to add",
                    },
                    "number2": {
                        "type": "number",
                        "description": "The second number to add",
                    },
                },
                "required": ["number1", "number2"],
            },
        },
    },

    {
        "type": "function",
        "function": {
            "name": "extract_code",
            "description": "Determines whether the provided text contains code. Use this tool whenever a user asks for a function, script, or programming-related response.",
            "parameters": {
                "type": "object",
                "properties": {
                    "model_output": {
                        "type": "string",
                        "description": "The text to analyze for the presence of code.",
                    },
                },
                "required": ["model_output"],
            },
        },
    }



]

In [3]:
# Assuming you have a list or registry of tools
# tools = [tool_definitions]  # Add your new tool to the list of tools
tools=tool_definitions

In [2]:
# from dotenv import load_dotenv
# import os
# import json
# import numpy as np
# import traceback

# # Load environment variables from .env file
# load_dotenv(dotenv_path=os.path.join(os.path.expanduser("~/Desktop"), "env"))

# # Get the OpenAI API key
# openai_api_key = os.getenv('OPENAI_API_KEY_DRACO')
# langsmith_api_key = os.getenv('LANGSMITH_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY_DRACO')
# together_api_key = os.getenv('TOGETHER_API_KEY_DRACO')
# open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
# gemini_api_key = os.getenv('GEMINI_API_KEY')
# judge_model='openai/gpt-4o-mini' #If used with Llama, only 0.01$ for 9 Q&A pairs for gpt-4o-mini, and 0.22$ for gpt-4o


In [14]:
def decide_tool_usage(query, tools=tools, judge_model=judge_model, openai_api_key=openai_api_key):
    """Decide if a tool should be used based on the query, and if yes, output the tool name(s)."""

    # Construct prompt for the judge
    tool_descriptions = "\n".join(
        f"Tool Name: {tool.function.name}\nDescription: {tool.function.description}\nParameters: {', '.join(tool.function.parameters['properties'].keys())}"
        for tool in tools
    )
    
    prompt = f"""Given a user question, determine if any tool from the provided list should be used to answer the question.
    Consider:
    1. The capability of each tool, based on its name, description, and parameters, to provide useful information for answering the question
    2. If using no tool might be better than using a potentially misleading tool

    User Question: {query}

    Available Tools:
    {tool_descriptions}

    Should a tool be used for answering the question? If yes, specify the tool name(s). Respond with 'No' or the tool name(s).
    """
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant that determines tool usage."},
        {"role": "user", "content": prompt}
    ]
    
    # Use OpenAI to judge tool usage
    import openai
    from langsmith.wrappers import wrap_openai
    client = wrap_openai(openai.Client(api_key=openai_api_key))
    
    response = client.chat.completions.create(
        messages=messages,
        temperature=0,
        model=judge_model.split('/')[1],
        seed=42
    )
    
    tool_decision = response.choices[0].message.content.strip()
    print("Tool Decision:", tool_decision)
    
    if tool_decision.lower() == 'no':
        return None
    else:
        return tool_decision.split(', ')

In [4]:
def decide_tool_usage(query, tools=tool_definitions, judge_model=judge_model, openai_api_key=openai_api_key):
    """Decide if a tool should be used based on the query, and if yes, output the tool name(s)."""

    # Construct prompt for the judge
    tool_descriptions = "\n".join(
        f"Tool Name: {tool['function']['name']}\nDescription: {tool['function']['description']}\nParameters: {', '.join(tool['function']['parameters']['properties'].keys())}"
        for tool in tools
    )
    
    prompt = f"""Given a user question, determine if any tool from the provided list should be used to answer the question.
    Consider:
    1. The capability of each tool, based on its name, description, and parameters, to provide useful information for answering the question
    2. If using no tool might be better than using a potentially misleading tool

    User Question: {query}

    Available Tools:
    {tool_descriptions}

    Should a tool be used for answering the question? If yes, specify the tool name(s). Respond with 'No' or the tool name(s).
    """
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant that determines tool usage."},
        {"role": "user", "content": prompt}
    ]
    
    # Use OpenAI to judge tool usage
    import openai
    from langsmith.wrappers import wrap_openai
    client = wrap_openai(openai.Client(api_key=openai_api_key))
    
    response = client.chat.completions.create(
        messages=messages,
        temperature=0,
        model=judge_model.split('/')[1],
        seed=42
    )
    
    tool_decision = response.choices[0].message.content.strip()
    print("Tool Decision:", tool_decision)
    
    if tool_decision.lower() == 'no':
        return None
    else:
        return tool_decision.split(', ')

In [5]:
print(decide_tool_usage('How much is 3^3?'))
print(decide_tool_usage('How much is 3+3?'))
print(decide_tool_usage("What's the weather in Rotterdam?"))
print(decide_tool_usage("Write a function to load a df and calculate the sum of rows"))


Tool Decision: No
None
Tool Decision: add
['add']
Tool Decision: No
None
Tool Decision: extract_code
['extract_code']


In [8]:
def execute_tool(tool_name, query):
    if tool_name=='extract_code':
        # Use OpenAI to judge tool usage
        import openai
        from langsmith.wrappers import wrap_openai
        client = wrap_openai(openai.Client(api_key=openai_api_key))

        messages = [
            {"role": "system", "content": "Return only the code from the following message to be directly copy pasted in a py file. Do not return it in quotes, just plain code. \
             The message is:"},
            {"role": "user", "content": query}
        ]
        
        response = client.chat.completions.create(
            messages=messages,
            temperature=0,
            model=judge_model.split('/')[1],
            seed=42
        )
        
        code_result = response.choices[0].message.content
        print("Type",type(code_result))
        print("Resulting code:",code_result)


        import subprocess

        # Define the command to execute the code in the 'test_LLM' conda environment
        command = f"conda run -n test_code_execution python -c \"{code_result}\""
        print("Command",command)

        try:
            # Execute the command and capture the output
            result = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT, text=True)
            print("Execution Result:", result)
        except subprocess.CalledProcessError as e:
            result = f"Error in execution: {e.output}"
            print(result)


        return code_result



    elif tool_name == 'calculator':
        # Example tool: calculator
        # This tool can perform basic arithmetic operations
        try:
            result = eval(query)
            return result
        except Exception as e:
            return f"Error in calculation: {e}"
    elif tool_name == 'weather':
        # Example tool: weather
        # This tool can fetch weather information for a given location
        location = query.split('in')[-1].strip()
        # Assuming we have a function get_weather_info(location) that fetches weather info
        try:
            result = get_weather_info(location)
            return result
        except Exception as e:
            return f"Error in fetching weather information: {e}"
    elif tool_name == 'add':
        try:
            import openai
            from langsmith.wrappers import wrap_openai
            client = wrap_openai(openai.Client(api_key=openai_api_key))

            messages = [
                {"role": "system", "content": "Return only the two numbers from the following message that have to be added in the format a+b. The message is:"},
                {"role": "user", "content": query}
            ]
            
            response = client.chat.completions.create(
                messages=messages,
                temperature=0,
                model=judge_model.split('/')[1],
                seed=42
            )
            
            add_result = response.choices[0].message.content
            print(add_result)

            numbers = add_result.split('+')
            number1 = float(numbers[0].strip())
            number2 = float(numbers[1].strip())
            result = number1 + number2
            print("Result of addition",result)
            return result
        except Exception as e:
            return f"Error in addition: {e}"

    # elif tool_name == 'code_generator':
    #     # Example tool: code_generator
    #     # This tool can generate code snippets based on the query
    #     try:
    #         # Assuming we have a function generate_code(query) that generates code
    #         result = generate_code(query)
    #         return result
    #     except Exception as e:
    #         return f"Error in code generation: {e}"
    else:
        return f"Tool {tool_name} is not recognized."

# Example usage
tool_name = decide_tool_usage('How much is 3^3?')
if tool_name:
    print(execute_tool(tool_name, '3^3'))
print("\n")

tool_name = decide_tool_usage('How much is 3 plus 3?')
if tool_name:
    if 'add' in tool_name:
        print(execute_tool('add', 'How much is 3 plus 3?'))
print("\n")

tool_name = decide_tool_usage('How much is 3 + 3?')
if tool_name:
    if 'add' in tool_name:
        print(execute_tool('add', 'How much is 3 + 3?'))
print("\n")

tool_name = decide_tool_usage("What's the weather in Rotterdam?")
if tool_name:
    print(execute_tool(tool_name, "What's the weather in Rotterdam?"))
print("\n")

tool_name = decide_tool_usage("Write a function to load a df. It should load the df 'test.xlsx' from desktop by dynamically getting the desktop path (os.path.expanduser('~')) and print it")
if tool_name:
    if 'extract_code' in tool_name:
        execute_tool('extract_code', "Write a function to load a df. It should load the df 'test.xlsx' from desktop by dynamically getting the desktop path (os.path.expanduser('~')) and print it")
         #not print here
print("\n")

Tool Decision: No


Tool Decision: add
3+3
Result of addition 6.0
6.0


Tool Decision: add
3+3
Result of addition 6.0
6.0


Tool Decision: No


Tool Decision: extract_code
Type <class 'str'>
Resulting code: import pandas as pd
import os

def load_dataframe():
    desktop_path = os.path.expanduser('~') + '/Desktop/test.xlsx'
    df = pd.read_excel(desktop_path)
    print(df)

load_dataframe()
Command conda run -n test_code_execution python -c "import pandas as pd
import os

def load_dataframe():
    desktop_path = os.path.expanduser('~') + '/Desktop/test.xlsx'
    df = pd.read_excel(desktop_path)
    print(df)

load_dataframe()"
Execution Result:                                     input          output
0                        How much is 3*3?               9
1  What is the result of 128905342*423423  54581486625666






In [17]:
#How the dataset will be named in Langsmith
def get_dataset_name(model_name, judge_model):
    # try: #For Hugging Face models
        if not any(provider in model_name for provider in commercial_api_providers): #For Hugging Face models
            return domain+"_Engineering_Evaluation_"+model_name.split('/')[1]+'_with_judge_'+judge_model+'_beam_search_statistics_all' #Chemical
        else: #For Commercial API providers
            return domain+"_Engineering_Evaluation_"+model_name+'_with_judge_'+judge_model+'_beam_search_statistics_all' #Chemical
    # except: #For OpenAI models
    #     return "Chemical_Engineering_Evaluation_"+model_name+'_with_judge_'+judge_model+'_beam_search_statistics_all'

Check if GPU is available

In [18]:
import torch
print(torch.version.cuda)

12.1


Google Drive mount (If run in Colab)

In [19]:
if 'content/drive/My Drive' in file_path:
    from google.colab import drive
    drive.mount('/content/drive')

Read Excel File

In [20]:
import pandas as pd
qa=pd.read_excel(file_path) #Read Excel
qa=qa[['input','output']]

Create Dataset from df

In [21]:
from datasets import Dataset
loaded_dataset=Dataset.from_pandas(qa)

if inference_on_whole_dataset==False:
    loaded_dataset = loaded_dataset.train_test_split(test_size=0.2, seed=42) #Used if going to fine-tune in part of the dataset

In [22]:
if inference_on_whole_dataset==False:
    dataset_train=loaded_dataset['train']
    dataset_test=loaded_dataset['test']
else:
    dataset_test=loaded_dataset #When we use the whole dataset

Create Langsmith Test Dataset

In [23]:
#https://docs.smith.langchain.com/old/evaluation/faq/manage-datasets

from langsmith import Client

example_inputs = [(x['input'],x['output']) for x in dataset_test]
print(example_inputs)

def create_langsmith_dataset(dataset_name, example_inputs, langsmith_api_key):

    client = Client(api_key=langsmith_api_key)

    try:
        #Load the dataset if already exists
        for existing_dataset in client.list_datasets():
            if existing_dataset.name==dataset_name:
                dataset_langsmith=existing_dataset
        for x in dataset_langsmith:
            print("Dataset Loaded")
            break

    except: #Otherwise create it
        print("Dataset not found. Creating new dataset")
        # Storing inputs in a dataset lets us run chains and LLMs over a shared set of examples.
        dataset_langsmith = client.create_dataset(dataset_name=dataset_name,
                                                description="Q&A"+ domain + "engineering.") #chemical

        for input_prompt, output_answer in example_inputs:
            client.create_example(
                inputs={"question": input_prompt.replace('\n', ' ')},
                outputs={"answer": output_answer.replace('\n', ' ')},
                # metadata={"source": "Wikipedia"},
                dataset_id=dataset_langsmith.id,
            )

    return dataset_langsmith

[('How are flow corrections applied around each loop in the Hardy Cross method, and why must flow continuity be maintained at every junction?', 'In the Hardy Cross method, one first selects individual loops in the network and makes an initial guess of the flow in each pipe. Then, each loop is corrected iteratively: the head loss around the loop is summed, and a flow correction is applied to reduce the total head-loss error to zero. This correction is added (or subtracted) to the assumed flow in every pipe forming that loop. Flow continuity must be maintained at every junction to ensure that all flow entering a node is accounted for by either leaving through connecting pipes or satisfying demand. If continuity is not enforced, the model would not reflect true network behavior, causing errors to propagate in subsequent iterations.'), ('Which factors influence the choice of initial flow guesses in the Hardy Cross approach, and how can poor choices affect convergence?', 'Typical choices fo

Custom Evaluation Metrics

In [24]:
# https://docs.smith.langchain.com/old/cookbook/introduction
# https://docs.smith.langchain.com/old/evaluation/faq/custom-evaluators
# https://docs.smith.langchain.com/how_to_guides/evaluation/evaluate_llm_application#use-a-summary-evaluator

from langsmith.schemas import Run, Example
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from termcolor import colored

list_of_metrics=['completeness_descr','relevance_descr','conciseness_descr','confidence_descr','factuality_descr','judgement_descr']

#Function that compares the real answer with the predicted answer of an LLM and returns a score based on the evaluation
def factor_evaluator(run: Run, example: Example) -> dict: 
    # print("Run:",run)

    question=run.inputs.get("inputs")['question']
    # print("Question:",question)
    actual_answer = example.outputs.get("answer")
    # print("Real answer:",example.outputs.get("answer"))
    predicted_answer = run.outputs.get("output")
    # print("Predicted Answer:",answer)

    scores={} #Store scores for each metric
    descriptions={} #Store descriptions for each metric
    
    # Check if there is output from LLM
    if not predicted_answer:
        print("No output from LLM")
        keys=[]
        for metric_name in list_of_metrics: #o1 sometimes doesn't give an answer
            keys.append(metric_name.split('_descr')[0])
            scores[metric_name]=0
            descriptions[metric_name]='-'
        results = {
            "results": [{"key": key, "score": scores[key + "_descr"], "value": descriptions[key + "_descr"]} for key in keys]}
        return results

    else:
        for metric_name in list_of_metrics: #Iterate through all metrics
            print("Evaluating based on:",metric_name)
            metric_value=common_prompt+eval(metric_name) #Get the actual description of the metric

            # Define roles and placeholders
            chat_template = ChatPromptTemplate.from_messages(
            [("system", metric_value),
                ("user", "Question: {question}, Actual answer: {actual_answer}, Predicted answer: {predicted_answer}"),
                # ("ai", "It's sunny and warm outside."), #Use this if we want to use few shot prompts
            ]
            )

            messages = chat_template.format_messages(question=question, actual_answer=actual_answer, predicted_answer=predicted_answer)
            # print("Messages:",messages)

            formatted_messages = [(role, msg.content) for role, msg in zip(["system", "user"], messages)]
            # print("Formatted messages:",formatted_messages) #[('system', 'You are an autoregressive lan....', 'user':.....)]

            # Initialize the model and get response
            llm = ChatOpenAI(model_name=judge_model.split('/')[1], api_key=openai_api_key, temperature=0, max_tokens=max_output_tokens, seed=42)
            ai_response = llm.invoke(formatted_messages)

            # Output
            # print(colored("System message:"+ messages[0].content,'blue'))
            print(colored("User message:"+ messages[1].content, 'green'))
            print(colored("AI message:"+ ai_response.content,'red'))

            #Decide what the final score is based on output
            if "FINAL SCORE:" in ai_response.content: 
                score = int(ai_response.content.split("FINAL SCORE:")[1])
            else:
                print("Invalid response from LLM:", ai_response.content)
                score = 0 #For cases where the LLM doesn't return a score - Otherwise we are gonna get an error

            scores[metric_name]=score
            descriptions[metric_name]=ai_response.content
            print("Scores:",scores)
            print("\n")

    return {
        "results":[ #We always need 'key', 'score' pairs
            {"key": "completeness" , "score": scores['completeness_descr'],"value":descriptions['completeness_descr']},
            {"key": "relevance" , "score": scores['relevance_descr'], "value":descriptions['relevance_descr']},
            {"key": "conciseness" , "score": scores['conciseness_descr'], "value":descriptions['conciseness_descr']},
            {"key": "confidence" , "score": scores['confidence_descr'], "value":descriptions['confidence_descr']},
            {"key": "factuality" , "score": scores['factuality_descr'], "value":descriptions['factuality_descr']},
            {"key": "judgement" , "score": scores['judgement_descr'], "value":descriptions['judgement_descr']}
        ]
    }

Define Models that Generate Responses

In [25]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.random.manual_seed(0) #Set for reproducibility

def initialize_model(model_id):
    # # Check if mps acceleration is available (For MacOS)
    # device = "mps" if torch.backends.mps.is_available() else "cpu"
    # print(f"Using device {device}")
    # model.to(device)

    # transformers.set_seed(42) #Tried for reproducibility but didn't work
    
    pipeline = transformers.pipeline( 
            "text-generation",
            model=model_id,
            model_kwargs={"torch_dtype": torch.bfloat16, "cache_dir":cache_dir},
            # trust_remote_code=True,
            device_map="auto" #Use 'cuda' if one GPU available (works in Delft Blue with 32GB VRAM) - 'auto' the alternative for distributed over all available GPUs
        )
    return pipeline

def get_model(model_id):
    """Given a model name, return the loaded model, tokenizer, and pipeline"""

    # if 'openai' not in model_id and 'groq_website' not in model_id and 'anthropic' not in model_id and 'together' not in model_id: #For Hugging Face models
    if not any(provider in model_id for provider in commercial_api_providers): #For Hugging Face models
        pipeline=initialize_model(model_id)

    #Returns below variables if defined, and returns None for any that are not.
    model = locals().get('model', None)
    tokenizer = locals().get('tokenizer', None)
    pipeline = locals().get('pipeline', None)

    return model, tokenizer, pipeline

Generate Responses

In [26]:
import time

def predict(inputs: dict) -> dict:
    """Given a question, return the answer from the model"""
    
    #Get these variables from the global scope
    global model_name, generate_max_tokens
    
    messages = [ #Only use the questions to ask the model to generate the response
      {"role": "user", "content": inputs['question']},
    ]

    #They do not support system message
    if 'gemma' not in model_name and 'anthropic' not in model_name and 'openrouter' not in model_name and 'gemini' not in model_name and '/o1' not in model_name: 
      messages.insert(0, {"role": "system", "content": "You are a language model specialized in "+ domain + " engineering. Answer the following question:"}) #chemical
    elif 'gemini' in model_name: #Google Gemini has this format since it also gets images
      messages = {"role": "user", "parts": [{"text": "You are a language model specialized in "+ domain + "engineering. Answer the following question: " + messages[0]['content']}]}
    else: #For gemma add system prompt in user message
      messages[0]['content']="You are a language model specialized in "+ domain + " engineering. Answer the following question: " + messages[0]['content']
    # print("Prompt:",messages)

    #Reasoning model with CoT should have longer max_tokens to include the reasoning steps - For now deepseek and Gemini Flash Thinking
    if 'deepseek' in model_name or 'thinking' in model_name or '/o1' in model_name or '/o3' in model_name or 'QwQ-32B-AWQ' in model_name:
      generate_max_tokens=4000
      print("Generation limit increased due to reasoning model:", model_name, "to:",generate_max_tokens)
    else: #Since it's global it might stay to 4000 otherwise
      generate_max_tokens=1000

    generation_args = { 
        "max_new_tokens": generate_max_tokens,
        "return_full_text": False, 
        "temperature": 0.05, #Has to be positive number - not considered from model when do_sample is False (reproducible results)
        "do_sample": True, #Selects highest probability token if sets to False
        "num_beams" : 5, #3 can also work if computationally intensive - more info on https://huggingface.co/blog/how-to-generate
        #Warnings will be raised by some models

        #If we only set temp!=0 or if we also set do_sample=False then warning: `do_sample` is set to `False`. However, `temperature` is set to `1e-08` 
        # -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
        # That means that the temperature is probably ignored
        # Sometimes, results not reproducible if only temp is set
        # A temparature of 0.01 or lower results in: "Error running target function: probability tensor contains either `inf`, `nan` or element < 0"
      } 
    
    if not any(provider in model_name for provider in commercial_api_providers): #For Hugging Face models
      response=pipeline(messages, **generation_args)[0]['generated_text']
      print(model_name,':',response)

    else: 
      if 'openai' in model_name:
        try:
          import openai
          from langsmith.wrappers import wrap_openai
                  
          # Define OpenAI client
          openai_client = wrap_openai(openai.Client(api_key=openai_api_key))
          
          if '/o1' not in model_name and '/o3' not in model_name:
            response = openai_client.chat.completions.create(messages=messages, temperature=0, model=model_name.split('/')[1], max_tokens=generate_max_tokens, seed=42) 
          else: #For thinking models
            print("Thinking....")
            response = openai_client.chat.completions.create(messages=messages, model=model_name.split('/')[1], max_completion_tokens=generate_max_tokens, seed=42) 

          # print("Full Response from OpenAI:",response)
          response=response.choices[0].message.content #That's the response without formatting
          print("Response from OpenAI:",response)
          time.sleep(5) #To avoid rate limiting

        except Exception as e:
          print("Error:",e)
          print("OpenAI Model ID:",model_name)

      elif 'groq_website' in model_name:
        try:
          from groq import Groq
          client = Groq()
          actual_model_name=model_name.split('/')[1]
          response = client.chat.completions.create(
              model=actual_model_name,
              max_tokens=generate_max_tokens,
              temperature=0,
              messages=messages)
          
          response=response.choices[0].message.content #That's the response without formatting
          print("Response from Groq:",response)#.choices[0].message.content)
          time.sleep(5) #To avoid rate limiting

        except Exception as e:
          print("Error:",e)
          print("Groq Model ID:",model_name)

      elif 'anthropic' in model_name:
        try:
          import anthropic
          client = anthropic.Anthropic()
          response = client.messages.create(
              model=model_name.split('/')[1],
              messages=messages,
              temperature=0,
              max_tokens=generate_max_tokens,
          )
          response=response.content[0].text #That's the response without formatting
          print("Response from Anthropic:",response)#.content[0].text)
          time.sleep(5) #To avoid rate limiting

        except Exception as e:
          print("Error:",e)
          print("Anthropic Model ID:",model_name)

      elif 'together' in model_name:
        try:
          from together import Together
          client = Together()
          response = client.chat.completions.create(
              model="/".join(model_name.split("/")[1:]),
              messages=messages,
              temperature=0,
              max_tokens=generate_max_tokens
          )
          # print("Full Response from Together:",response_full)
          response=response.choices[0].message.content #That's the response without formatting
          print("Response from Together:",response)
          time.sleep(5) #To avoid rate limiting
          if "<think>" in response:
            time.sleep(180) #To avoid rate limiting need to wait 3 minutes

        except Exception as e:
          print("Error:",e)
          print("Together Model ID:",model_name)
      
      elif 'openrouter' in model_name: #Very limited quota for free models
        try:
          from openai import OpenAI

          client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=open_router_api_key,
          )
          response = client.chat.completions.create(
            model="/".join(model_name.split("/")[1:]),
            messages=messages,
            temperature=0,
            max_tokens=generate_max_tokens,
          )
          response=response.choices[0].message.content #That's the response without formatting
          print("Response from OpenRouter:",response)#.choices[0].message.content)
          time.sleep(5) #To avoid rate limiting

        except Exception as e:
          print("Error:",e)
          print("OpenRouter Model ID:",model_name)

      elif 'gemini' in model_name:
        try:
          if 'thinking' in model_name: #Thinking model has different call
            from google.generativeai.types import GenerationConfig
            import google.generativeai as genai
            model = genai.GenerativeModel(model_name.split('/')[1])
            response = model.generate_content(
                contents=messages,
                generation_config=GenerationConfig(
                    temperature=0,
                    max_output_tokens=generate_max_tokens,
                )
            )
            print("Full Response from Gemini ('thinking') model:",response)
            response=response.text
            print("Response from Gemini ('thinking') model:",response)
            time.sleep(5) #To avoid rate limiting
          else: #for the rest of the models
            from google import genai
            from google.genai import types
            client = genai.Client(api_key=gemini_api_key)
            response = client.models.generate_content(
              model=model_name.split('/')[1],
              contents=messages,
              config=types.GenerateContentConfig(
                  temperature=0,
                  max_output_tokens=generate_max_tokens,
              )
            )
            response=response.text
            print("Response from Gemini:",response)
            time.sleep(5) #To avoid rate limiting
        except Exception as e:
          print("Error:",e)
          print("Gemini Model ID:",model_name)


    return {"output": response}

In [27]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

def calculate_statistics(values):
    """Calculate mean, standard error, and confidence intervals."""
    mean_value = np.mean(values)  # Mean of the metric over single run and over single metric (but over all questions)
    std_error = np.std(values, ddof=1) / np.sqrt(len(values))  # ddof=1 to divide by n-1 to calculate the sample sd
    
    assert np.std(values, ddof=1) == np.sqrt(np.sum((values-mean_value)**2)/(len(values)-1)), "Standard deviation calculation mismatch"
    
    margin_of_error = 1.96 * std_error  # didn't use t_critical=t.ppf(0.975, df=len(values)-1) since we're using sample standard deviation

    return {
        'mean': mean_value,
        'std_error': std_error,
        'ci_low': mean_value - margin_of_error,
        'ci_high': mean_value + margin_of_error
    }

def plot_metric_distributions(metric_values, axes, colors, bin_edges, metric_names):
    """Plot individual metric distributions with error bars."""
    error_bars = []
    run_stats = {}
    
    for metric_idx, (metric_name, values) in enumerate(metric_values.items()):  # Loop over runs' metric names and values
        clean_metric_name = metric_name.replace('_descr', '')  # This is over one run and over one metric (but over all questions)
        metric_name = metric_names[metric_idx]
        assert clean_metric_name == metric_name, "Metric name mismatch"
        
        stats = calculate_statistics(values)
        sns.histplot(values, bins=bin_edges, color=colors[metric_idx], ax=axes[metric_idx], kde=False)
        
        #Store error bars
        if metric_idx == 0:
            error_bars = []
        error_bars.append((stats['mean'], axes[metric_idx].get_ylim()[1]/2, stats['ci_high'] - stats['mean']))
        
        run_stats[metric_name] = stats

        axes[metric_idx].set_title(f'{metric_name} (Mean: {stats["mean"]:.2f} ± {stats["std_error"]:.2f} SE, CI: {stats["ci_low"]:.2f}-{stats["ci_high"]:.2f})')
        axes[metric_idx].set_xlim(0, 5.5)  # Keep 0 in case of errors
        axes[metric_idx].set_ylabel('Frequency')
        axes[metric_idx].set_xlabel('Values' if metric_idx == len(metric_values)-1 else '')
        
    return error_bars, run_stats

def plot_question_scores(metric_names, grouped_values, colors):
    """Plot scores for each question across metrics."""
    
    plt.figure(figsize=(10, 6))

    # Define colors for each metric
    colors = plt.cm.Set3(np.linspace(0, 1, len(metric_names)))

    # First count all frequencies per score (1-5) per metric for one run over all questions
    question_scores_by_metric = {metric: [] for metric in metric_names}
    score_metric_counts = {}

    #Plot each metric's values and store question scores
    for i, (metric, question_scores) in enumerate(zip(metric_names, grouped_values)):
        width = 0.8 / len(question_scores)  # Width of each metric's bar
        
        for j, val in enumerate(question_scores): #Create a bar for each question's score
            plt.bar(i + j * width, val, width=width, color=colors[i], alpha=0.5, 
                    label=metric if j == 0 else "")
                    # i is the index of metric and determines the base position of a group of bars corresponding to that metric.
                    # j*width adds an offset to the base position to separate individual bars within the same group (metric). 
                    # Each j corresponds to a different value in question_scores, creating distinct bars for the values of question_scores for the same metric.
                    # By combining the above two, we get the exact x-position of a specific bar     
            question_scores_by_metric[metric].append((j, val))

        counts = Counter(question_scores)  # Count frequency of each score in question_scores (e.g. {4: 1, 3: 2, 2: 2, 1: 1, 0: 1}, where key is score)
        for score, freq in counts.items():
            if score not in score_metric_counts:
                score_metric_counts[score] = {}
            score_metric_counts[score][metric] = freq  #Keeps track of how many times each metric gets a specific score over all questions (for one run)
            # {4: {'completeness': 1, 'confidence': 1, 'factuality': 1, 'judgement': 1}, 3: {'completeness': 1, 'relevance': 2, 'conciseness': 2, ....}

    return question_scores_by_metric, score_metric_counts

def plot_ordered_scores(metric_names, question_scores_by_metric, colors):
    """Plot metrics ordered by score values."""
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metric_names):
        plt.subplot(len(metric_names), 1, i+1)
        sorted_questions = sorted(question_scores_by_metric[metric], key=lambda x: x[1]) #Sort questions by score
        
        #Plot bars
        x_pos = range(len(sorted_questions))
        scores = [q[1] for q in sorted_questions] #q[1] is the score, q[0] is the index
        plt.bar(x_pos, scores, color=colors[i], alpha=0.5)

        #Add question indices as x-axis labels
        plt.xticks(x_pos, [str(q[0]) for q in sorted_questions])
        
        plt.ylabel(metric)
        plt.ylim(0, 5.5)
        plt.yticks(range(6))  # Set y-axis ticks from 0 to 5

        if i == len(metric_names)-1:
            plt.xlabel('Question number (ordered by score)')

def plot_accumulated_distributions(score_metric_counts, metric_names, colors):
    """Plot accumulated distribution of scores by metric."""
    legend_added = set()

    #For each score, plot metrics in order of frequency (highest frequency at bottom)
    for score in sorted(score_metric_counts.keys()):
        #Sort metrics by frequency for this score
        sorted_metrics = sorted(score_metric_counts[score].items(),
                            key=lambda x: x[1], #Use the frequency (second element of each tuple) as the sorting key
                            reverse=True)  # highest frequency first
        bottom = 0
        for metric, freq in sorted_metrics:
            i = metric_names.index(metric) #get index for color
            plt.bar(score, freq,
                    width=0.4,
                    color=colors[i],
                    alpha=0.5,
                    label=metric if metric not in legend_added else "",
                    bottom=bottom)
            bottom += freq
            legend_added.add(metric)

           
def plot_figures_metrics(all_runs_model_metrics, metric_names, model_name, judge_model):
    """
    Creates visualizations and calculates statistics for evaluation metrics across multiple runs.

    Args:
        all_runs_model_metrics (dict): Nested dictionary containing evaluation metrics for each model and run.
            Structure: {model_id: [{metric1_descr_run1: [q1_score, q2_score, ...], 
                                  metric2_descr_run1: [q1_score, q2_score, ...], ...}, 
                                 {metric1_descr_run2: [q1_score, q2_score, ...],
                                  metric2_descr_run2: [q1_score, q2_score, ...], ...},
                                 ...num_runs]}
            Example: {'model1': [{'completeness_descr_run1': [4.5, 3.0, 4.0], 
                                'relevance_descr_run1': [3.5, 4.0, 3.0]}, ...,
                               {'completeness_descr_run2': [4.0, 3.5, 4.5],
                                'relevance_descr_run2': [3.0, 4.5, 3.5], ...},
                               ...num_runs]}
            Where each inner dictionary represents one run containing scores for each metric across all questions
        metric_names (list): Names of metrics to analyze and plot (e.g. ['completeness', 'relevance'])
        model_name (str): Name/identifier of the model being evaluated
        judge_model (str): Name/identifier of the model used for judging the evaluations

    Returns:
        dict: Summary statistics for each model, run and metric.
            Structure: {model_name: {run_idx: {metric_name: {
                'mean': float,
                'std_error': float, 
                'ci_low': float,
                'ci_high': float
            }}}}
            Example: {'anthropic/claude-3-5-sonnet': {
                '0': {'completeness': {'mean': 4.5, 'std_error': 0.5, 
                                     'ci_low': 3.52, 'ci_high': 5.48},
                      'relevance': {'mean': 3.5, 'std_error': 0.5,
                                  'ci_low': 2.52, 'ci_high': 4.48} , ...},
                '1': {'completeness': {'mean': 4.5, 'std_error': 0.5,
                                     'ci_low': 3.52, 'ci_high': 5.48},
                      'relevance': {'mean': 3.5, 'std_error': 0.5,
                                  'ci_low': 2.52, 'ci_high': 4.48}, ...},
                ...num_runs}}

    The function generates several visualization types:
    - Individual histograms for each metric showing score distributions
    - Error bars indicating means and confidence intervals
    - Overlapping bar plots comparing metrics
    - Stacked distribution plots showing relative frequencies of scores

    All plots are saved as PNG files with names indicating the judge model,
    evaluated model, run index, and plot type.
    """

    summary_stats_all_runs = {}  # Keep track of summary statistics over all runs

    for run_idx, metric_values_run in enumerate(all_runs_model_metrics[model_name]): #Loop over runs

        colors = sns.color_palette("Set3", len(metric_names))
        
        # Create two figures - one with separate subplots and one overlaid
        fig, axes = plt.subplots(len(metric_names), 1, figsize=(10, 18))
        plt.subplots_adjust(hspace=0.6, top=0.94)
        fig.suptitle(f'Metric Distributions for {model_name} (Run {run_idx})', fontsize=16)
        
        bin_edges = np.arange(0.0, 5.6, 0.2)  # Bins for range 0-5
        metric_names = [name.replace('_descr', '') for name in metric_values_run]
        
        error_bars, run_stats = plot_metric_distributions(metric_values_run, axes, colors, bin_edges, metric_names)
        
        # Save version without error bars
        plt.figure(fig.number)
        plt.savefig(f"{judge_model.split('/')[1]}_judge_with_{model_name.replace('/', '_')}_run_{run_idx}_metric_distributions_no_error_bars.png")
        
        # Add error bars and save updated version
        for i, (mean, ylim, margin) in enumerate(error_bars):
            axes[i].errorbar(mean, ylim, xerr=margin, color='black', capsize=5, 
                           capthick=1, elinewidth=2, marker='o')
        
        plt.savefig(f"{judge_model.split('/')[1]}_judge_with_{model_name.replace('/', '_')}_run_{run_idx}_metric_distributions.png")
        plt.close('all')

        # Print summary statistics - Can also be seen in txt file. 
        print(f"\nSummary Statistics over run {run_idx}:")
        print("-" * 50)
        for metric, stats in run_stats.items():
            print(f"{metric}:")
            for key, value in stats.items():
                print(f"  {key}: {value:.2f}")
            print("-" * 50)

        summary_stats_all_runs[run_idx] = run_stats #For one run

        grouped_values=list(metric_values_run.values()) #Values of all metrics for one run over all questions. There are num_metrics lists in that list. 
        values = [val for sublist in grouped_values for val in sublist] #Flatten the list - Size is num_questions*num_metrics (1st metric questions, 2nd metric questions, etc)
        
        question_scores_by_metric, score_metric_counts = plot_question_scores(metric_names, grouped_values, colors)
        plt.xlabel('Metrics')
        plt.ylabel('Score')
        plt.title('Per-Metric Question Scores Distribution')
        plt.xticks(np.arange(len(metric_names)) + 0.1, metric_names)
        plt.yticks(range(6)) #Set y-ticks to 0-5
        plt.savefig(f"{judge_model.split('/')[1]}_judge_with_{model_name.replace('/', '_')}_run_{run_idx}_per_metric_question_scores.png")
        plt.close('all')

        # Plot ordered scores
        plot_ordered_scores(metric_names, question_scores_by_metric, colors)
        plt.suptitle('Question indices ordered by metric value')
        plt.tight_layout()
        plt.savefig(f"{judge_model.split('/')[1]}_judge_with_{model_name.replace('/', '_')}_run_{run_idx}_question_indices_ordered_by_metric_value.png")
        plt.close('all')

        # Plot accumulated distributions
        plt.figure(figsize=(10, 6))
        plot_accumulated_distributions(score_metric_counts, metric_names, colors)
        plt.xlabel('Score')
        plt.ylabel('Frequency')
        plt.title('Score Distribution Histogram by Metric') 
        plt.legend(title='Metrics', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(np.arange(0, 6))
        plt.tight_layout()
        plt.savefig(f"{judge_model.split('/')[1]}_judge_with_{model_name.replace('/', '_')}_run_{run_idx}_score_distribution_histogram_by_metric.png")
        plt.close('all')

    return summary_stats_all_runs

Perform the Evaluation over all models

In [28]:
#https://python.langchain.com/v0.2/docs/integrations/chat/openai/
from langsmith.evaluation import evaluate

def load_model_stats(judge_model): #In case we had to restart the loop - some models didn't run - Keep track of all model stats
    """Load existing stats from files or initialize empty dictionaries."""
    try:
        with open(f'stats_{judge_model.split("/")[1]}.json', 'r') as f:
            all_models_stats = json.load(f)
    except FileNotFoundError:
        all_models_stats = {}  # Used in comparison between models

    try:
        with open(f'all_runs_model_metrics_{judge_model.split("/")[1]}.json', 'r') as f:
            all_runs_model_metrics = json.load(f)
    except FileNotFoundError:
        all_runs_model_metrics = {}  # Used in plotting metrics
        
    return all_models_stats, all_runs_model_metrics

def perform_evaluation(model_id, judge_model, n_resamples, example_inputs, factor_evaluator, langsmith_api_key):
    """Perform evaluation runs and collect results."""
    dataset_name = get_dataset_name(model_id, judge_model) #How the dataset will be named in Langsmith
    dataset_langsmith = create_langsmith_dataset(dataset_name, example_inputs, langsmith_api_key)

    evaluation_all_resamples = [] #Used below to obtain the unique questions/answers and also the results of each resample
    
    begin = time.time()
    for resample_idx in range(n_resamples):
        print(f"\nPerforming evaluation of resample {resample_idx+1}/{n_resamples} of {model_id}")

        evaluation_results = evaluate(
            predict, #Function that call our LLM and returns its output
            data=dataset_langsmith.name, #Just using dataset_langsmith doesn't work 
            evaluators=[factor_evaluator], #Evaluators to use
            max_concurrency=1, #Run one question through langsmith each time - Other values will give errors in resulting excels
            # metadata={"revision_id": "the version of your pipeline you are testing"},
            experiment_prefix=str(judge_model)+'_judge_with_'+str(model_id)+'_resample_'+str(resample_idx) # A prefix for your experiment names to easily identify them
        )
        evaluation_all_resamples.extend(evaluation_results) #Used below to get unique questions/answers and to select the predicted answers
        #This has n_resamples*num_questions elements, for just one model

    with open('evaluation_all_resamples_'+str(model_id.split('/')[1])+'_'+str(judge_model.split('/')[1])+'.txt', 'w') as f:
        f.write(str(evaluation_all_resamples))

    assert len(evaluation_all_resamples)==n_resamples*len(example_inputs), f"Number of evaluation results not matching num_resamples*num_questions. \
        Got {len(evaluation_all_resamples)} evaluation results but expected {n_resamples*len(example_inputs)}"
    
    print(f"Total time for evaluation: {time.time() - begin}")

    return evaluation_all_resamples, dataset_langsmith

def process_evaluation_results(langsmith_api_key, dataset_langsmith):
    """Extract questions and answers from evaluation results."""
    #https://docs.smith.langchain.com/tutorials/Developers/evaluation

    # Get unique questions/answers
    client = Client(api_key=langsmith_api_key)
    questions_answers=[x for x in client.list_examples(dataset_id=dataset_langsmith.id)]
    list_of_questions=[x.inputs['question'] for x in questions_answers]
    list_of_answers=[x.outputs['answer'] for x in questions_answers]
        
    # with open('list_of_questions.txt', 'w') as f:
    #     f.write(str(list_of_questions))
        
    # with open('list_of_answers.txt', 'w') as f:
    #     f.write(str(list_of_answers))
    
    results_df = pd.DataFrame({
        'questions': list_of_questions,
        'answers': list_of_answers
    })
    return results_df, list_of_questions

def process_metrics(resample_results, list_of_metrics, list_of_questions, resample_idx, results_df, model_name):
    """
    Process metrics for a single resample and update results DataFrame.
    
    Args:
        resample_results: Results from current resample
        list_of_metrics: List of metrics to process
        resample_idx: Current resample index
        results_df: DataFrame to update with metrics
        model_name: Name of the model being evaluated
        
    Returns:
        individual_run_metric_scores, metrics, results_df
    """

    metrics = [] #This should be the same as resample_results (list) except when there are 'traceback' errors where it will be 0.
    # metrics format will be:[[EvaluationResult(key='completeness', score=4, value='To evaluate the .... - It has num_questions sublists, each with num_metrics values

    for result in resample_results:
        if result['run'].outputs['output'] is None or not result['evaluation_results']['results']: #or result['run'].error is not None - Same as first condition
            metrics.append(0)  # Use 0 to indicate failed evaluation - We might even get in here when LangSmith API connection issues
            print("Error: No metric value found!")
            #Also print which condition is true
            print("result['run'].outputs['output'] is None",result['run'].outputs['output'] is None)
            print("not result['evaluation_results']['results']",not result['evaluation_results']['results'])
            # print("result['run'].error is not None",result['run'].error is not None)
        else:
            metrics.append(result['evaluation_results']['results'])

    with open('resample_results_'+str(resample_idx)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
        f.write(str(resample_results))

    assert len(resample_results)==len(list_of_questions), f"Number of resample results not matching num_questions. Got {len(resample_results)} resample \
        results but expected {len(list_of_questions)}"
    
    with open('metrics_'+str(resample_idx)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
        f.write(str(metrics))

    assert len(metrics)==len(list_of_questions), f"Number of metrics not matching num_questions. Got {len(metrics)} metrics but expected {len(list_of_questions)}"
    
    #This is at the end a dict with num_metrics keys and each key has num_questions values.
    #Example: {'completeness_descr': [4, 3, 3, 5, 5, 4, 3], 'relevance_descr': [4, 3, 3, 3, 4, 3, 1], ....} assuming 7 questions
    individual_run_metric_scores = {} #Keep track of scores of all metrics over all questions for one resample

    for metric_idx, metric_name in enumerate(list_of_metrics): #Get specific metric name and values over all questions for the current resample

        clean_metric_names, metric_scores, metric_prompts = [], [], [] #Metric scores and prompts for all questions for a given resample - Should be num_questions elements each time
        
        #Get all metric keys for the current resample over all questions, handling potential missing keys (values set to 0 for those - they are errors)
        for m in metrics:
            if m == 0: #If there is an error
                key = metric_name.replace('_descr','')
                score = 0
                prompt=""
            else:
                try:
                    key = m[metric_idx].key #Metric name
                    score = m[metric_idx].score ##Scores of a given metric over all questions for a given resample
                    prompt = m[metric_idx].value #Prompt used for the evaluation
                except:
                    print("Error: Metric not found - Shouldn't get here")
                    key = metric_name.replace('_descr','')
                    score = 0
                    prompt = ""
                
            clean_metric_names.append(key)
            metric_scores.append(score)
            metric_prompts.append(prompt)
            
        assert all(name == metric_name.replace('_descr','') for name in clean_metric_names), f"Metric keys not matching: clean_metric_names={clean_metric_names}, \
            expected={metric_name.replace('_descr','')} and their values: {metric_scores}"
            
        # with open('metric_scores_'+str(resample_idx)+'_'+str(metric_name)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
        #     f.write(str(metric_scores))

        assert len(metric_scores)==len(list_of_questions), f"Number of metric scores not matching num_questions. Got {len(metric_scores)} metric scores \
            but expected {len(list_of_questions)}"
            
        # with open('metric_prompts_'+str(resample_idx)+'_'+str(metric_name)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
        #     f.write(str(metric_prompts))

        assert len(metric_prompts)==len(list_of_questions), f"Number of metric prompts not matching num_questions. Got {len(metric_prompts)} metric prompts \
            but expected {len(list_of_questions)}"

        # Update results DataFrame
        clean_metric_name = clean_metric_names[0] #Just one metric name without the _descr
        results_df[f'metric_{clean_metric_name}_{resample_idx+1}'] = metric_scores
        results_df[f'prompt_{clean_metric_name}_{resample_idx+1}'] = metric_prompts
        
        # Store scores for return
        individual_run_metric_scores[metric_name] = metric_scores #len is num_metrics
    
    # with open('individual_run_metric_scores_'+str(resample_idx)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
    #     f.write(str(individual_run_metric_scores))

    return individual_run_metric_scores, metrics, results_df

def calculate_metric_statistics(all_runs_metric_scores, list_of_metrics, num_questions, model_name):
    """Calculate statistical metrics across resamples (reduce variance - step 3.1)."""
    metric_stats_resampling = {} # Calculate mean and standard error for each metric and question across K resamples
    #The above dict will have num_metrics elements, each with metric keys (e.g. mean, std, etc), that will have num_questions values
    #Example: {'completeness': {'mean':[4, 3, 3, 5, 5, 4, 3]}, #here for a dataset with 7 questions
    #          'relevance': {'mean':[4, 3, 3, 3, 4, 3, 2]}, ...}
    
    for metric in list_of_metrics:
        metric_stats_resampling[metric] = {
            'means': [],  # Mean score across K resamples for each question
            'standard_errors': [],  # Standard error of the mean for each question
            'conditional_vars': []  # Conditional variance reduced by factor of K
        }
        
        # For each question
        for q in range(num_questions):
            # Get K scores for this metric/question across all resamples (num_resamples elements each time in that list)
            scores = [run[metric][q] for run in all_runs_metric_scores]
            K = len(scores)  # Number of resamples
            assert len(scores)==n_resamples, f"Number of scores not matching num_resamples. Got {len(scores)} scores but expected {n_resamples}"
            
            # Calculate statistics
            mean = np.mean(scores) #Average score of each question for a given metric over all resamples
            var = np.var(scores) #Variance of the scores of each question for a given metric over all resamples
            # Calculate conditional variance reduced by factor of K. Var(mean) = σ²/K where σ² is the variance of individual scores
            conditional_var = var / K if K > 0 else 0
            standard_error = np.sqrt(conditional_var)
            
            # Store results
            metric_stats_resampling[metric]['means'].append(mean)
            metric_stats_resampling[metric]['standard_errors'].append(standard_error)
            metric_stats_resampling[metric]['conditional_vars'].append(conditional_var)
    
    with open('metric_stats_resampling_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
        f.write(str(metric_stats_resampling))

    assert len(metric_stats_resampling)==len(list_of_metrics), f"Number of metric_stats_resampling not matching num_metrics. \
        Got {len(metric_stats_resampling)} metric_stats_resampling but expected {len(list_of_metrics)}"
    
    for metric in list_of_metrics:
        assert len(metric_stats_resampling[metric]['means']) == num_questions, f"Number of values for metric '{metric}' ({len(metric_stats_resampling[metric]['means'])}) \
            not matching expected number of questions ({num_questions})"

    return metric_stats_resampling

def handle_zero_values(results_df, n_resamples, list_of_metrics):
    """
    Handle zero values in results.
    
    Args:
        results_df (pd.DataFrame): DataFrame containing results
        n_resamples (int): Number of resamples
        list_of_metrics (list): List of metrics to check
        
    Returns:
        dict: Indices of rows containing zero values for each metric
    """
    zero_rows_columns = {}
    
    try:
        # Handle 0 values across all resamples - These are errors
        for resample_idx in range(n_resamples):
            for metric in list_of_metrics:
                try:
                    simple_metric_name = metric.replace('_descr','')
                    metric_col = f'metric_{simple_metric_name}_{resample_idx+1}'
                    
                    # Check if column exists
                    if metric_col not in results_df.columns:
                        print(colored(f"Warning: Column {metric_col} not found in DataFrame", 'yellow'))
                        continue
                    
                    zero_indices = results_df[metric_col] == 0 #series with True/False
                    
                    if zero_indices.any(): #If any of the values of that column are 0
                        zero_rows_columns[metric_col] = []
                        for idx in zero_indices[zero_indices].index: #Loop over True indices (rows with 0s)
                            try:
                                print(colored(f"Missing value for metric '{simple_metric_name}' in resample {resample_idx+1}", 'red'))
                                print(colored(f"Question: {results_df.loc[idx, 'questions']}", 'green'))
                                zero_rows_columns[metric_col].append(idx) #Keep track of columns and rows with zero values
                            except Exception as e:
                                print(colored(f"Unexpected error processing zero value at row {idx}: {e}", 'red'))
                
                except Exception as e:
                    print(colored(f"Error processing metric {metric} in resample {resample_idx}: {e}", 'red'))
        
        return zero_rows_columns # Return column names and rows with zero values
    
    except Exception as e:
        print(colored(f"Critical error in handle_zero_values: {e}", 'red'))
        traceback.print_exc()
        return {}  # Return empty dict in case of critical error

def process_zero_values(results_df, zero_rows_columns, list_of_metrics, model_name): #TO BE ACTIVATED
    """Process and optionally replace zero values in results."""
    row_zero_counts = {}
    col_zero_counts = {}

    # Force a copy to ensure changes are applied properly below when replace with mean value
    results_df_copy = results_df.copy()

    for column_name, row_indices in zero_rows_columns.items():
        for row_idx in row_indices:
                
            # Get values for this metric for this row and column (one resample per time)
            values = results_df.loc[row_idx, column_name]

            assert values==0, "Values should be 0"
            
            if values != 0: #We should never get here
                with open('values_'+str(model_name.split('/')[1])+'_'+str(column_name)+'_'+str(row_idx)+'.txt', 'w') as f:
                    f.write(str(values))
                
            #Given that values are 0, replace with mean of non-zero values
            df_values=results_df.loc[:, column_name].values
            non_zero_values = [x for x in df_values if x != 0]

            if len(non_zero_values) > 0:
                mean_value = np.mean(non_zero_values)

                if results_df.loc[row_idx, column_name] == 0 and mean_value != 0:
                    print(colored(f"0 value in row {row_idx}, column {column_name} should be replaced with mean {mean_value:.2f}", 'yellow'))
                    # Uncomment to actually replace values:
                    # results_df.loc[row_idx, column_name] = mean_value#round(mean_value, 1)

                    row_zero_counts[row_idx] = row_zero_counts.get(row_idx, 0) + 1
                    col_zero_counts[column_name] = col_zero_counts.get(column_name, 0) + 1

    print("\nZero values replaced per row:")
    for row in sorted(row_zero_counts):
        print(f"Row/question {row}: {row_zero_counts[row]} replacements")

    print("\nZero values replaced per column:")
    for col in sorted(col_zero_counts):
        print(f"Column/metric {col}: {col_zero_counts[col]} replacements")

def reorganize_evaluation_metrics(all_resamples_metrics, list_of_metrics, model_name, list_of_questions, n_resamples):
    """    
    This function takes evaluation metrics from multiple resampling runs and reorganizes them into
    a structured dictionary where each metric's scores are grouped together. It handles cases where
    some evaluations may have failed (represented by 0s).
    
    Args:
        all_resamples_metrics (list): List of evaluation results for each resample. Each resample contains
                                     scores for multiple questions and metrics.
        list_of_metrics (list): List of metric names to process (e.g., ['completeness_descr', 'relevance_descr']).
        model_name (str): Name of the model being evaluated, used for logging.
        list_of_questions (list): List of questions that were evaluated.
        n_resamples (int): Number of resampling iterations performed.
    
    Returns:
        dict: Dictionary where keys are metric names (without '_descr' suffix) and values are lists
              containing all scores for that metric across all resamples and questions.
              
    Note:
        The function assumes each resample has scores for all questions and metrics.
    """
    metric_scores_all_resamples = {metric.replace('_descr', ''): [] for metric in list_of_metrics}
    #The above dict will have num_metrics elements, with their value for each question, over each run (first num_questions for first run, then next num_questions for next run, etc)
    #Example: {'completeness': {'mean':[4, 3, 3, 5, 5, 4, 3, 4, 3, 3, 5, 5, 0, 3, 5, 3, 3, 5, 5, 4, 3]},  #assuming 3 runs and 7 questions
    #          'relevance': {'mean':[4, 3, 3, 3, 4, 3, 2, 4, 3, 3, 3, 3, 0, 2, 4, 3, 3, 3, 3, 3, 2]}, ...}
    #In case of error, there will be num_questions less elements in the sublist for which there was an error
    
    for metric_name in list_of_metrics:
        clean_name = metric_name.replace('_descr', '')
        
        #Each resample_metrics (num_resamples in total) has a list of num_questions lists, each having num_metrics values
        #format of each sublist: [EvaluationResult(key='completeness', score=4, value='To evaluate the ...
        #If error, instead of the above list we have just a 0.
        for resample_idx, resample_metrics in enumerate(all_resamples_metrics):

            with open('resample_metrics_'+str(resample_idx)+'_'+str(metric_name)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
                f.write(str(resample_metrics))

            metric_idx = list_of_metrics.index(metric_name) #0-num_metrics the range of values of this. 

            scores = [m[metric_idx].score if m!=0 and m!=[] else 0 
                     for m in resample_metrics] #num_questions elements each time
            assert len(scores)==len(list_of_questions), "Scores length not matching num_questions"

            with open('scores_'+str(resample_idx)+'_'+str(metric_name)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
                f.write(str(scores))

            metric_scores_all_resamples[clean_name].extend(scores) #Every time we add one metric for one resample (num_questions elements)

            with open('metric_stats_reorganized_'+str(resample_idx)+'_'+str(metric_name)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
                f.write(str(metric_scores_all_resamples))

    assert [len(x) for x in metric_scores_all_resamples.values()]==[len(list_of_questions)*n_resamples]*len(list_of_metrics), "Metric stats length not matching"

    return metric_scores_all_resamples

def save_results(results_df, judge_model, model_id, stage="before"):
    """Save results DataFrame to Excel."""
    filename = (f"results_{judge_model.split('/')[1]}_judge_with_"
               f"{model_id.replace('/','_')}_{stage}_nan_replacement.xlsx")
    
    try:
        #Extract reasoning traces and final answers from predicted answers
        # Check for <think> tags in predicted answer columns and split them if found
        for col in results_df.columns:
            if 'predicted_answer' in col:
                # Create new column names
                run_number = col.split('_')[-1]
                reasoning_col = f'reasoning_trace_{run_number}'
                
                # Check if we need to split this column
                has_think_tags = results_df[col].astype(str).str.contains('</think>', na=False).any()
                
                if has_think_tags:
                    print("Has </think> in answer")
                    # Extract reasoning traces and final answers
                    reasoning_traces = []
                    final_answers = []
                    
                    for answer in results_df[col]:
                        if isinstance(answer, str):
                            if '<think>' in answer and '</think>' in answer:
                                # Extract the reasoning trace between <think> and </think>
                                think_start = answer.find('<think>') + len('<think>')
                                think_end = answer.find('</think>')
                                
                                if think_start >= 0 and think_end >= 0:
                                    reasoning = answer[think_start:think_end].strip()
                                    print("Reasoning:",reasoning)
                                    final_answer = answer[think_end + len('</think>'):].strip()
                                    print("Final answer:",final_answer)
                                    reasoning_traces.append(reasoning)
                                    final_answers.append(final_answer)
                            elif '</think>' in answer:
                                print("Only </think> in answer")
                                # Handle case where only </think> is present
                                think_end = answer.find('</think>')
                                reasoning = answer[:think_end].strip()
                                print("Reasoning:",reasoning)
                                final_answer = answer[think_end + len('</think>'):].strip()
                                print("Final answer:",final_answer)
                                reasoning_traces.append(reasoning)
                                final_answers.append(final_answer)
                            else:
                                reasoning_traces.append('')
                                final_answers.append(answer)
                    
                    if any(trace.strip() for trace in reasoning_traces): #If there is any reasoning trace
                        # Add the new columns - insert reasoning column right after the predicted answer column
                        col_idx = results_df.columns.get_loc(col)
                        results_df.insert(col_idx + 1, reasoning_col, reasoning_traces)
                        results_df[col] = final_answers
                        print("Added reasoning traces and final answers to dataframe")

    except Exception as e:
        print("Error in saving trace results:", e)
        traceback.print_exc()
    
    results_df.to_excel(filename, index=False)

In [29]:
# Main execution loop (move code below inside this function)
# def main():
all_models_stats, all_runs_model_metrics = load_model_stats(judge_model) #Try to load already saved data (if some models have already been evaluated), otherwise initialize empty dicts

for model_id in models:
    global model_name, model, tokenizer, pipeline, generate_max_tokens
    model, tokenizer, pipeline = get_model(model_id)
    model_name = model_id #Since model_name defined as global variable
    
    try: #Sometimes some errors with the evaluation
        evaluation_all_resamples, dataset_langsmith = perform_evaluation(model_id, judge_model, n_resamples, example_inputs, factor_evaluator, langsmith_api_key)
        chunk_size = len(example_inputs) #Number of questions
        results_df, list_of_questions = process_evaluation_results(langsmith_api_key, dataset_langsmith)
        
        all_resamples_metrics = [] #Keep track of all metrics over all resamples and all questions
        #There will be n_resamples lists, each with num_questions sublists (each having num_metrics sublists) (so num_questions*num_metrics elements in those in total)
        #Each question will have 6 metric values like this: [EvaluationResult(key='completeness', score=4, value='To evaluate the ....
        all_runs_metric_scores = [] #This will be appended to the input that plots metrics at the end. 
        #The format of it is [{metric1_descr_run1: [q1_score, q2_score, ...], metric2_descr_run1: [q1_score, q2_score, ...], ...}, 
        #                     {metric1_descr_run2: [q1_score, q2_score, ...], metric2_descr_run2: [q1_score, q2_score, ...], ...},
        #                     ...num_runs]
        
        # Process each resample
        for resample_idx in range(n_resamples):
            start_idx = resample_idx * chunk_size #start index of current resample (chunk size is the number of questions of each resample)
            #Resample_results saved above in the process_metrics function
            resample_results = evaluation_all_resamples[start_idx:start_idx + chunk_size] #Get results of a particular resample
            assert len(resample_results)==chunk_size, f"Number of resample results not matching num_questions. Got {len(resample_results)} resample results but expected {chunk_size}"

            predicted_answers = [x['run'].outputs['output'] for x in resample_results] #None if error
            assert len(predicted_answers)==chunk_size, f"Number of predicted answers not matching num_questions. Got {len(predicted_answers)} predicted answers but expected {chunk_size}"

            # with open('predicted_answers_'+str(resample_idx)+'_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
            #     f.write(str(predicted_answers))

            #Add predicted answers to df
            results_df[f'predicted_answer_{resample_idx+1}'] = predicted_answers

            individual_run_metric_scores, metrics, results_df = process_metrics(
                    resample_results, 
                    list_of_metrics, 
                    list_of_questions,
                    resample_idx,
                    results_df,
                    model_name
                )           

            # Handle zero values
            zero_rows_columns = handle_zero_values(results_df, n_resamples, list_of_metrics)
            if zero_rows_columns:
                unique_zero_rows_columns = len(set([x for sublist in list(zero_rows_columns.values()) for x in sublist]))
                print(colored(f"ERROR: Found missing values in {unique_zero_rows_columns} rows out of {len(results_df)}", 'red'))
                process_zero_values(results_df, zero_rows_columns, list_of_metrics, model_name) #Replace 0s with mean of non-zero values     
            
            #In each iteration we append the metrics (6 in total) of one resample for all questions - n at the end, one for each resample
            #If there is an error, the metrics will be 0 (there will be n_errors*num_metrics less 'EvaluationResult' objects in that case)
            all_resamples_metrics.append(metrics)

            #Has n_resamples lists, each with num_metrics sublists (each sublist has scores over all questions of one metric) 
            all_runs_metric_scores.append(individual_run_metric_scores)
        
        assert len(all_runs_metric_scores)==n_resamples, f"Number of all_runs_metric_scores not matching num_resamples. \
            Got {len(all_runs_metric_scores)} all_runs_metric_scores but expected {n_resamples}"
        
        for i in range(n_resamples):
            assert len(all_runs_metric_scores[i])==len(list_of_metrics), f"Number of all_runs_metric_scores[{i}] not matching num_metrics. \
                Got {len(all_runs_metric_scores[i])} all_runs_metric_scores[{i}] but expected {len(list_of_metrics)}"

        with open('all_runs_metric_scores_main_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
            f.write(str(all_runs_metric_scores))

        with open('all_resamples_metrics_main_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
            f.write(str(all_resamples_metrics))

        assert len(all_resamples_metrics)==n_resamples, f"Number of all_resamples_metrics not matching num_resamples. \
            Got {len(all_resamples_metrics)} all_resamples_metrics but expected {n_resamples}"
        
        for i in range(n_resamples): #Each one will have num_questions elements, each with num_metrics sublists (or 0 if error)
            assert len(all_resamples_metrics[i])==len(list_of_questions), f"Number of all_resamples_metrics[{i}] not matching num_questions. \
                Got {len(all_resamples_metrics[i])} all_resamples_metrics[{i}] but expected {len(list_of_questions)}" #Each all_ressamples_metrics[i] should have num_questions elements

        # Calculate statistics
        metric_stats_resampling = calculate_metric_statistics(
            all_runs_metric_scores, 
            list_of_metrics, 
            len(list_of_questions),
            model_name
        )
        
        # Save initial results
        save_results(results_df, judge_model, model_id, "before")
        
        # # Handle zero values
        # zero_rows_columns = handle_zero_values(results_df, n_resamples, list_of_metrics)
        # if zero_rows_columns:
        #     unique_zero_rows_columns = len(set([x for sublist in list(zero_rows_columns.values()) for x in sublist]))
        #     print(colored(f"ERROR: Found missing values in {unique_zero_rows_columns} rows out of {len(results_df)}", 'red'))
        #     process_zero_values(results_df, zero_rows_columns, list_of_metrics, model_name) #Replace 0s with mean of non-zero values
        
        # Reorganize metrics - Has num_metrics keys, each with num_questions*num_resamples values (as a list)
        metric_scores_all_resamples = reorganize_evaluation_metrics(all_resamples_metrics, list_of_metrics, model_name, list_of_questions, n_resamples)

        with open('metric_scores_all_resamples_final_main_'+str(model_name.split('/')[1])+'.txt', 'w') as f:
            f.write(str(metric_scores_all_resamples))

        assert len(metric_scores_all_resamples)==len(list_of_metrics), f"Number of metric_scores_all_resamples not matching num_metrics. \
            Got {len(metric_scores_all_resamples)} metric_scores_all_resamples but expected {len(list_of_metrics)}"
        
        for i in range(len(list_of_metrics)):
            name_of_metric=list_of_metrics[i].replace('_descr','')
            assert len(metric_scores_all_resamples[name_of_metric])==len(list_of_questions)*n_resamples, f"Number of metric_scores_all_resamples[{name_of_metric}] not matching \
                num_questions*num_resamples. Got {len(metric_scores_all_resamples[name_of_metric])} metric_scores_all_resamples[{name_of_metric}] but \
                expected {len(list_of_questions)*n_resamples}"

        metric_names = list(metric_scores_all_resamples.keys()) #Final list of metrics for plotting
        
        # Verify metric names
        metrics_names_loop = [metric.replace('_descr','') for metric in list_of_metrics]
        assert metrics_names_loop == metric_names, "Metric names mismatch"
        
        # Save results
        all_runs_model_metrics[model_id] = all_runs_metric_scores #Used in plotting metrics
        #Dictionary in format {model_id:[{metric_1_run_1:[values], metric_2_run_1:[values], ...}, {metric_1_run_2:[values]....}]

        all_models_stats[model_id] = plot_figures_metrics(
            all_runs_model_metrics,
            metric_names,
            model_id,
            judge_model
        ) #Stats like mean, std, etc. per metric and per run over all questions
        
        # Save to files
        with open(f'stats_{judge_model.split("/")[1]}.json', 'w') as f:
            json.dump(all_models_stats, f, indent=4)
        with open(f'all_runs_model_metrics_{judge_model.split("/")[1]}.json', 'w') as f:
            json.dump(all_runs_model_metrics, f, indent=4)

        print("Model",model_id,"saved")
        print("Models saved so far:",list(all_models_stats.keys()))
            
    except Exception as e:
        print("An error occurred in evaluating model",model_id)
        print("Error Details:", e)
        traceback.print_exc()
    
    finally:
        # Clear VRAM
        del model, tokenizer, pipeline
        torch.cuda.empty_cache()
        print('-'*100)

# if __name__ == "__main__":
#     main()

Loading checkpoint shards: 100%|██████████| 4/4 [00:38<00:00,  9.52s/it]
Device set to use cuda:0


Dataset not found. Creating new dataset

Performing evaluation of resample 1/1 of meta-llama/Meta-Llama-3.1-8B-Instruct
View the evaluation results for experiment: 'openai/gpt-4o-mini_judge_with_meta-llama/Meta-Llama-3.1-8B-Instruct_resample_0-cff151da' at:
https://smith.langchain.com/o/42866633-791c-5d79-833d-4e624439c57c/datasets/8788f428-b783-4a27-9014-5899ab20823e/compare?selectedSessions=b341e826-1d91-4593-89c0-1de9d2daf231




0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


meta-llama/Meta-Llama-3.1-8B-Instruct : Computational modeling of very large water networks is a complex task that poses several challenges. Some of the key challenges include:

1. **Scalability**: As the size of the network increases, the computational time and memory requirements also increase exponentially, making it difficult to solve the problem within a reasonable time frame.

2. **Complexity**: Very large water networks often involve complex interactions between different components, such as pumps, valves, and pipes, which can lead to non-linear behavior and make it challenging to develop accurate models.

3. **Uncertainty and variability**: Real-world water networks are subject to various uncertainties and variability, such as changes in demand, pipe roughness, and valve settings, which can affect the accuracy of the model.

4. **Data quality and availability**: Large water networks often have limited data availability, and the data that is available may be of poor quality, mak

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

User message:Question: What aspects of the Newton-Raphson technique generally lead to faster convergence compared to loop-based iterative methods?, Actual answer: Newton-Raphson methods solve all governing equations simultaneously by using partial derivatives (the Jacobian) that link changes in flows or heads to changes in hydraulic equations. This approach allows the solver to make a more direct beeline toward the solution, whereas loop-based methods adjust flows one loop at a time. By incorporating global information into each iteration, Newton-Raphson typically reaches high accuracy in fewer iterations, especially for large or complex systems., Predicted answer: The Newton-Raphson technique generally leads to faster convergence compared to loop-based iterative methods due to the following aspects:

1. **Quadratic Convergence**: The Newton-Raphson method exhibits quadratic convergence, meaning that the number of correct digits in the solution approximately doubles with each iteration

33it [17:05, 31.24s/it]

User message:Question: How are flow corrections applied around each loop in the Hardy Cross method, and why must flow continuity be maintained at every junction?, Actual answer: In the Hardy Cross method, one first selects individual loops in the network and makes an initial guess of the flow in each pipe. Then, each loop is corrected iteratively: the head loss around the loop is summed, and a flow correction is applied to reduce the total head-loss error to zero. This correction is added (or subtracted) to the assumed flow in every pipe forming that loop. Flow continuity must be maintained at every junction to ensure that all flow entering a node is accounted for by either leaving through connecting pipes or satisfying demand. If continuity is not enforced, the model would not reflect true network behavior, causing errors to propagate in subsequent iterations., Predicted answer: In the Hardy Cross method, flow corrections are applied around each loop to satisfy the continuity equation

33it [18:06, 32.92s/it]


Total time for evaluation: 1089.2825152873993
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not result['evaluation_results']['results'] False
Error: No metric value found!
result['run'].outputs['output'] is None True
not 

Statistical comparison between models

In [30]:
def compare_model_performances(all_runs_model_metrics): #all_models_stats, 
    """
    Performs statistical comparison between models using paired differences, standard errors,
    and Pearson correlation coefficients following section 4.2 methodology.
    
    Args:
        all_models_stats (dict): Dictionary containing statistics for each model
        all_runs_model_metrics (dict): Dictionary containing raw metrics for each model/run/question
        
    Returns:
        dict: Dictionary containing pairwise comparison results
    """
    import numpy as np
    from scipy import stats
    import itertools
    
    # Get all model pairs for comparison
    models = list(all_runs_model_metrics.keys())
    model_pairs = list(itertools.combinations(models, 2))
    
    # Store results
    comparison_results = {}
    
    for model1, model2 in model_pairs:
        comparison_key = f"{model1.split('/')[-1]}_vs_{model2.split('/')[-1]}"
        comparison_results[comparison_key] = {}
        
        # Get metrics (removing '_descr' suffix)
        metrics = [metric.replace('_descr', '') for metric in list(all_runs_model_metrics[model1][0].keys())]
        
        # Create file for this model comparison
        variance_results_text = f"\n=== Variance Analysis Results for {comparison_key} ===\n"
        
        for metric in metrics:
            # Calculate differences and correlations for each resample
            resample_differences = []
            resample_ses = []
            correlations = []
            model1_variances = []  # Initialize list
            model2_variances = []  # Initialize list
            
            # Iterate through resamples - Same number for both models
            for resample_idx in range(len(all_runs_model_metrics[model1])):
                # Get scores for both models for this resample
                scores1 = all_runs_model_metrics[model1][resample_idx][f'{metric}_descr']
                scores2 = all_runs_model_metrics[model2][resample_idx][f'{metric}_descr']
                
                # Calculate differences for each question
                question_differences = np.array(scores1) - np.array(scores2)
                
                # Calculate mean difference for this resample
                mean_diff = np.mean(question_differences) #Same as the formula in the paper since mean(a-b)=mean(a)-mean(b)
                
                # Calculate standard error for this resample - Paired analysis (section 4.2)
                n = len(question_differences)
                se = np.sqrt(np.sum((question_differences - mean_diff)**2) / (n * (n-1))) if n > 1 else np.nan

                # # Calculate standard errors for each model - Unpaired analysis (section 4.1)
                # n = len(scores1)
                # sea = np.sqrt(np.sum((scores1 - np.mean(scores1))**2) / (n * (n - 1))) if n > 1 else np.nan
                # seb = np.sqrt(np.sum((scores2 - np.mean(scores2))**2) / (n * (n - 1))) if n > 1 else np.nan

                # # Calculate the combined standard error as sqrt(sea^2 + seb^2)
                # se = np.sqrt(sea**2 + seb**2)

                # Calculate variances for each model
                var1 = np.var(scores1, ddof=1)  # Using ddof=1 for sample variance
                var2 = np.var(scores2, ddof=1)
                model1_variances.append(var1)
                model2_variances.append(var2)
                
                # Calculate Pearson correlation
                correlation, _ = stats.pearsonr(scores1, scores2)
                
                resample_differences.append(mean_diff)
                resample_ses.append(se)
                correlations.append(correlation)
            
            # Convert to numpy arrays
            resample_differences = np.array(resample_differences)
            resample_ses = np.array(resample_ses)
            correlations = np.array(correlations)
            model1_variances = np.array(model1_variances)
            model2_variances = np.array(model2_variances)
            print("resample_differences",resample_differences)
            print("resample_ses",resample_ses)
            print("correlations",correlations)
            print(f"Model 1 variances: {model1_variances}")
            print(f"Model 2 variances: {model2_variances}")
          
            # Calculate overall mean difference over all resamples
            overall_mean_diff = np.mean(resample_differences)
            print("overall_mean_diff",overall_mean_diff)
            
            #We want an aggregated SE across all resamples for the same questions (same paired differences)
            #This approach accounts for the fact that each resampling provides a different estimate of the variance of the same underlying distribution, 
            # and averaging these estimates gives a better representation of the overall uncertainty.

            # Calculate pooled standard error across resamples
            R = len(resample_differences)
            pooled_se = np.sqrt(np.sum(resample_ses**2) / (R**2))
            print("pooled_se",pooled_se)
            
            # # If the resampling results are independent estimates of variance (i.e., combining uncertainty estimates from independent sources), the combined variance is
            # # the sum of all individual variances, and the combined standard error is given below (goal to capture total variability)
            # # Calculate the overall combined SE across all resamples
            # combined_se = np.sqrt(np.nansum(np.array(resample_ses)**2))

            # Calculate overall variance reduction across all resamples
            n = len(scores1)
            
            # Calculate mean variances across resamples
            mean_var1 = np.mean(model1_variances)  # Var(sA)
            mean_var2 = np.mean(model2_variances)  # Var(sB)
            
            # Calculate mean correlation across resamples
            mean_correlation = np.mean(correlations)
            
            # Calculate covariance between model scores
            mean_cov = mean_correlation * np.sqrt(mean_var1 * mean_var2)  # Cov(sA, sB)
            
            # Calculate variance for unpaired case: Var(μA-B,unpaired) = (Var(sA) + Var(sB))/n
            var_unpaired = (mean_var1 + mean_var2) / n
            
            # Calculate variance for paired case: Var(μA-B,paired) = (Var(sA) + Var(sB) - 2Cov(sA,sB))/n
            var_paired = (mean_var1 + mean_var2 - 2 * mean_cov) / n
            
            # The reduction in variance is: Var(μA-B,unpaired) - Var(μA-B,paired) = 2Cov(xA,xB)/n
            variance_reduction = 2 * mean_cov / n  # This should equal var_unpaired - var_paired
            
            # Calculate percentage reduction in variance
            percent_reduction = (variance_reduction / var_unpaired) * 100 if var_unpaired != 0 else 0

            # Add results for this metric to the text
            variance_results_text += f"\nMetric: {metric}\n"
            variance_results_text += f"Mean Model 1 variance (Var(sA)): {mean_var1:.6f}\n"
            variance_results_text += f"Mean Model 2 variance (Var(sB)): {mean_var2:.6f}\n"
            variance_results_text += f"Mean covariance (Cov(sA,sB)): {mean_cov:.6f}\n"
            variance_results_text += f"Unpaired variance: {var_unpaired:.6f}\n"
            variance_results_text += f"Paired variance: {var_paired:.6f}\n"
            variance_results_text += f"Variance reduction (2Cov(xA,xB)/n): {variance_reduction:.6f}\n"
            variance_results_text += f"Percent reduction: {percent_reduction:.1f}%\n"

            # # Calculate t-statistic and p-value
            # t_stat = overall_mean_diff / pooled_se if pooled_se != 0 else np.nan
            # df = R - 1  # degrees of freedom
            # p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df)) if not np.isnan(t_stat) else np.nan
            
            # # Calculate confidence interval
            # t_crit = stats.t.ppf(0.975, df)  # 95% CI
            # ci_margin = t_crit * pooled_se

            # Calculate z-statistic and CI using standard normal distribution
            z_stat = overall_mean_diff / pooled_se if pooled_se != 0 else np.nan
            
            # Calculate confidence interval using 1.96 for 95% CI
            ci_margin = 1.96 * pooled_se
            
            # Calculate p-value using standard normal distribution
            #For a two-tailed test p = 2 × (1 − Φ(|z|)), where Φ(z) is the cumulative distribution function (CDF) of the standard normal distribution.
            p_value = 2 * (1 - stats.norm.cdf(abs(z_stat))) if not np.isnan(z_stat) else np.nan
            
            # # Calculate average Pearson correlation - not accurate when correlations close to 1 or -1, variances differences across resamples, sample size is small.
            # avg_correlation = np.mean(correlations)

            #Apply Fisher z-transformation
            z_values = [0.5 * np.log((1 + r) / (1 - r)) for r in correlations]

            # Compute the mean Fisher z-value
            z_mean = np.mean(z_values)

            #Back-transform to Pearson correlation scale
            overall_correlation = (np.exp(2 * z_mean) - 1) / (np.exp(2 * z_mean) + 1)
            
            # Store results
            comparison_results[comparison_key][metric] = {
                "mean_difference": overall_mean_diff,
                "pooled_standard_error": pooled_se,
                "ci_low": overall_mean_diff - ci_margin,
                "ci_high": overall_mean_diff + ci_margin,
                # "t_statistic": t_stat,
                "z_statistic": z_stat,
                "p_value": p_value,
                "significant": p_value < 0.05 if not np.isnan(p_value) else None,
                "better_model": model1.split('/')[-1] if overall_mean_diff > 0 else model2.split('/')[-1],
                "pearson_correlation": overall_correlation
            }
        
        # Write all metrics results for this model comparison to a single file
        with open(f'variance_results_{comparison_key}.txt', 'w') as f:
            variance_results_text += f"Overall Variance Reduction Analysis:\n"
            f.write(variance_results_text)
    
    return comparison_results

comparison_results = compare_model_performances(all_runs_model_metrics) #all_models_stats

# Save results to file
with open('comparison_results.json', 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    def convert_to_serializable(obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.bool_)):
            return bool(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        elif obj is None:
            return None
        return obj
    
    serializable_results = json.loads(
        json.dumps(comparison_results, default=convert_to_serializable)
    )
    json.dump(serializable_results, f, indent=4)

In [31]:
# Plot comparison results
import matplotlib.pyplot as plt
import seaborn as sns

# Extract metrics and models from comparison_results
metrics = [metric.replace('_descr', '') for metric in list_of_metrics]
model_pairs = list(comparison_results.keys())

# Create figure with subplots for each metric
fig, axes = plt.subplots(2, 3, figsize=(25, 20), dpi=600)  # Increased DPI for higher resolution
fig.suptitle('Model Comparison Results by Metric', fontsize=16, y=1.05)
axes = axes.flatten()

for i, metric in enumerate(metrics):
    ax = axes[i]
    
    # Extract data for this metric
    means = []
    cis = []
    labels = []
    
    for pair in model_pairs:
        metric_data = comparison_results[pair][metric]
        means.append(metric_data['mean_difference'])
        # ci_margin = metric_data['ci_margin']
        cis.append([metric_data['ci_low'], 
                   metric_data['ci_high']])
        labels.append(pair)

    # Create bar plot
    bars = ax.bar(range(len(means)), means)
    
    # Add error bars for confidence intervals
    ax.errorbar(range(len(means)), means, 
               yerr=[[m - ci[0] for m, ci in zip(means, cis)],
                     [ci[1] - m for m, ci in zip(means, cis)]],
               fmt='none', color='black', capsize=5)
    
    # Add horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # Customize plot
    ax.set_title(f'{metric.capitalize()}')
    ax.set_xticks(range(len(means)))
    ax.set_xticklabels(labels, rotation=90) # Changed to vertical labels
    ax.set_ylabel('Mean Difference')
    
    # Color bars based on statistical significance
    for j, bar in enumerate(bars):
        if comparison_results[model_pairs[j]][metric]['p_value'] < 0.05:
            bar.set_color('darkred')
        else:
            bar.set_color('lightgray')

plt.tight_layout()

# Save plot before showing with high resolution
plt.savefig('model_comparisons.png', bbox_inches='tight', dpi=600)  # Increased DPI for higher resolution

# Show plot after saving
plt.show()

Create Tables

In [32]:
def create_comparison_table(comparison_results, metrics):
    """
    Creates a formatted table from comparison results.
    
    Args:
        comparison_results (dict): The comparison results dictionary
        metrics (list): List of metrics to include
        
    Returns:
        str: Formatted markdown table
    """
    # Table header
    table = "| Metric | Model | Baseline | Model - Baseline | 95% Conf. Interval | Correlation |\n"
    table += "|--------|--------|-----------|-----------------|-------------------|-------------|\n"
    
    # Add rows for each comparison and metric
    for pair in comparison_results:
        model1, model2 = pair.split('_vs_')
        for metric in metrics:
            results = comparison_results[pair][metric]
            
            row = f"| {metric} | {model1} | {model2} | "
            row += f"{results['mean_difference']:.1%} | "
            row += f"({results['ci_low']:.1%}, {results['ci_high']:.1%}) | "
            row += f"{results['pearson_correlation']:.2f} |\n"
            
            table += row
            
    return table

# Create and print the table
metrics = [m.replace('_descr', '') for m in list_of_metrics]
comparison_table = create_comparison_table(comparison_results, metrics)
print(comparison_table)

# Save table to file
with open('comparison_table.txt', 'w') as f:
    f.write(comparison_table)

| Metric | Model | Baseline | Model - Baseline | 95% Conf. Interval | Correlation |
|--------|--------|-----------|-----------------|-------------------|-------------|



Power Analysis

In [33]:
from statsmodels.stats.power import TTestIndPower

def perform_power_analysis(effect_size=0.5, alpha=0.05, power=0.8):
    """
    Perform power analysis to determine required sample size.
    
    Args:
        effect_size (float): Expected effect size (Cohen's d)
        alpha (float): Significance level
        power (float): Desired statistical power
        
    Returns:
        int: Required sample size per group
    """
    analysis = TTestIndPower()
    sample_size = analysis.solve_power(
        effect_size=effect_size,
        alpha=alpha,
        power=power,
        alternative='two-sided'
    )
    return int(np.ceil(sample_size))

# First, determine required sample size
required_samples = perform_power_analysis(effect_size=0.1254, alpha=0.05, power=0.8)  #These parameters result in a sample size of 1000
print(f"Required samples per model for statistical power: {required_samples}")

Required samples per model for statistical power: 1000


For real-time inference (below implementation only for meta-llama/Meta-Llama-3.1-8B-Instruct)

In [34]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# # del pipeline #Otherwise too much memory is used

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name,device_map='auto')

# #Example of real-time response generation
# messages=[{"role": "user", "content": "What is the chemical formula of water?"}]

# inputs_tokenized = tokenizer.apply_chat_template(
#     messages,
#     tokenize=True,
#     add_generation_prompt=True,
#     return_tensors="pt",
#     return_dict=True,
# ).to("cuda")

# input_ids = inputs_tokenized['input_ids']

# # Generate tokens one by one
# max_length = 256
# output_ids = input_ids
# for _ in range(256):
#     outputs = model.generate(
#         output_ids,
#         max_new_tokens=1,
#         do_sample=True,
#         top_k=50,
#         pad_token_id=tokenizer.eos_token_id
#     )
#     new_token_id = outputs[0, -1].item()
#     if new_token_id == tokenizer.eos_token_id:
#         break
#     output_ids = torch.cat([output_ids, outputs[:, -1:]], dim=1)
#     new_token = tokenizer.decode(new_token_id, skip_special_tokens=True)
#     print(new_token, end="", flush=True)

# print()

Other evaluators from Langsmith

In [35]:
# https://docs.smith.langchain.com/old/evaluation/faq/evaluator-implementations
# https://docs.smith.langchain.com/old/evaluation/quickstart

# from langsmith.evaluation import LangChainStringEvaluator

# eval_llm = ChatOpenAI(model_name=judge_model.split('/')[1], api_key=openai_api_key, temperature=0.0, seed=42)

# #Evaluators
# qa_evaluator = LangChainStringEvaluator("qa", config={"llm": eval_llm}) #LLM just gives 'correct' or 'incorrect' based on reference answer
# context_qa_evaluator = LangChainStringEvaluator("context_qa", config={"llm": eval_llm}) #Also uses reference context of example outputs to do the above
# cot_qa_evaluator = LangChainStringEvaluator("cot_qa", config={"llm": eval_llm}) #Same as above but with chain of thought 'reasoning'

#Prompts Used internally:

# 1) context_qa_evaluator: You are a teacher grading a quiz.
# You are given a question, the context the question is about, and the student's answer. You are asked to score the student's answer as either CORRECT or INCORRECT, 
# based on the context.

# Example Format:
# QUESTION: question here
# CONTEXT: context the question is about here
# STUDENT ANSWER: student's answer here
# GRADE: CORRECT or INCORRECT here

# Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. 
# It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 


# 2) cot_qa_evaluator: You are a teacher grading a quiz.
# You are given a question, the context the question is about, and the student's answer. You are asked to score the student's answer as either CORRECT or INCORRECT, 
# based on the context.
# Write out in a step by step manner your reasoning to be sure that your conclusion is correct. Avoid simply stating the correct answer at the outset.

# Example Format:
# QUESTION: question here
# CONTEXT: context the question is about here
# STUDENT ANSWER: student's answer here
# EXPLANATION: step by step reasoning here
# GRADE: CORRECT or INCORRECT here

# Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer.
#  It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 


# 3) qa_evaluator: You are a teacher grading a quiz.
# You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

# Example Format:
# QUESTION: question here
# STUDENT ANSWER: student's answer here
# TRUE ANSWER: true answer here
# GRADE: CORRECT or INCORRECT here

# Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer.
#  It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

Alternatively, use custom prompts as shown below (and set {"prompt": PROMPT} as additional argument inside the config above)

In [36]:
# from langchain_core.prompts.prompt import PromptTemplate

# _PROMPT_TEMPLATE = """You are an expert professor specialized in chemical engineering answers to questions.
# You are grading the following question:
# {query}
# Here is the real answer:
# {answer}
# You are grading the following predicted answer:
# {result}
# Respond with CORRECT or INCORRECT:
# """

# PROMPT = PromptTemplate(
#     input_variables=["query", "result", "answer"], template=_PROMPT_TEMPLATE
# )

Notes: Non-reproducible results, even when seed set (https://platform.openai.com/docs/api-reference/chat/create#chat-create-seed), temperature=0 (top_p should not change when we changed temperature - smaller values result in more constrained and focused response - https://medium.com/@rasithbm/chatopenai-parameters-83bef49f6384)